In [ ]:
!pip install faiss-cpu --quiet
!pip install google-generativeai --quiet
!pip install requests --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.7 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pickle
import faiss
import requests
import time
import threading
import tracemalloc
import google.generativeai as genai


from typing import List, Dict, Any
from google.colab import userdata

In [ ]:
# Configurações globais
MISTRAL_API_KEY = userdata.get('MISTRAL_KEY')
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
EMBEDDINGS_DIR = "/content/drive/MyDrive/Colab Notebooks/projeto II V3/scrapyFiles/faiss_index"
BASE_WAIT_TIME = 2.0
GEMINI_TIMEOUT = 30.0  # Timeout for Gemini API in seconds
GEMINI_MAX_RETRIES = 3
MAX_HISTORY_LENGTH = 10  # Limite de trocas no histórico de conversa

In [ ]:
# Inicializar o Gemini
genai.configure(api_key=GEMINI_API_KEY)


In [ ]:
# Testar a API do Gemini
print("Testando conexão com a API do Gemini...")
try:
    test_model = genai.GenerativeModel('gemini-2.0-flash-lite')
    test_response = test_model.generate_content("Quem descobriu o Brasil?")
    print(f"Teste bem-sucedido: {test_response.text[:100]}...")
except Exception as e:
    print(f"Erro ao testar a API do Gemini: {str(e)}")

Testando conexão com a API do Gemini...
Teste bem-sucedido: A descoberta do Brasil é tradicionalmente atribuída a Pedro Álvares Cabral, em 22 de abril de 1500. ...


In [ ]:
def measure_execution(func):
    """Decorator para medir tempo de execução e uso de memória."""
    def wrapper(*args, **kwargs):
        start_time = time.time()
        tracemalloc.start()
        result = func(*args, **kwargs)
        current, peak = tracemalloc.get_traced_memory()
        end_time = time.time()
        metrics = {
            "execution_time": end_time - start_time,
            "memory_used": current / 10**6,
            "memory_peak": peak / 10**6,
        }
        print(f"Tempo de execução: {metrics['execution_time']:.2f} segundos")
        print(f"Memória usada: {metrics['memory_used']:.2f} MB")
        print(f"Memória máxima: {metrics['memory_peak']:.2f} MB")
        tracemalloc.stop()
        return result, metrics
    return wrapper

def initialize_retriever(embeddings_dir: str):
    """Inicializa o sistema de retrieval."""
    print("Carregando recursos de retrieval...")
    try:
        embeddings = np.load(os.path.join(embeddings_dir, "book_embeddings.npy"))
        print("Embeddings carregados.")
        with open(os.path.join(embeddings_dir, "book_metadata.pkl"), "rb") as f:
            metadata = pickle.load(f)
        print("Metadados carregados.")
        index = faiss.read_index(os.path.join(embeddings_dir, "book_index.faiss"))
        print(f"Recursos carregados: {len(metadata)} livros indexados")
        return embeddings, metadata, index
    except Exception as e:
        print(f"Erro ao carregar recursos: {str(e)}")
        raise Exception(f"Falha na inicialização: {str(e)}")

def get_embedding(text: str) -> np.ndarray:
    """Gera embedding usando a API Mistral."""
    print(f"Gerando embedding para: {text}")
    headers = {"Authorization": f"Bearer {MISTRAL_API_KEY}", "Content-Type": "application/json"}
    url = "https://api.mistral.ai/v1/embeddings"
    payload = {"model": "mistral-embed", "input": text}

    for attempt in range(3):
        try:
            response = requests.post(url, json=payload, headers=headers, timeout=10)
            if response.status_code == 200:
                print("Embedding gerado com sucesso.")
                return np.array(response.json()["data"][0]["embedding"]).astype('float32').reshape(1, -1)
            elif response.status_code == 429:
                wait_time = BASE_WAIT_TIME * (2 ** attempt)
                print(f"Rate limit excedido. Aguardando {wait_time:.2f} segundos...")
                time.sleep(wait_time)
            else:
                print(f"Erro na API Mistral: {response.status_code} - {response.text}")
                raise Exception(f"Erro na API Mistral: {response.status_code} - {response.text}")
        except requests.Timeout:
            print(f"Tempo limite excedido na tentativa {attempt + 1}.")
            time.sleep(BASE_WAIT_TIME)
        except Exception as e:
            print(f"Exceção na tentativa {attempt + 1}: {str(e)}")
            time.sleep(BASE_WAIT_TIME)
    raise Exception("Falha ao gerar embedding após 3 tentativas.")

def search(query: str, index, metadata, top_k: int = 5) -> List[Dict[str, Any]]:
    """Busca livros similares."""
    print(f"Iniciando busca para: {query}")
    try:
        query_embedding = get_embedding(query)
        distances, indices = index.search(query_embedding, top_k)
        print("Busca concluída no índice FAISS.")
        results = [
            {
                "id": metadata[idx]["id"],
                "titulo": metadata[idx]["titulo"],
                "autor": metadata[idx]["autor"],
                "relevance_score": float(1.0 / (1.0 + dist)),
                "index": int(idx)
            }
            for idx, dist in zip(indices[0], distances[0])
        ]
        return results
    except Exception as e:
        print(f"Erro na busca: {str(e)}")
        raise

def format_context(results: List[Dict[str, Any]], metadata: List[Dict[str, Any]]) -> str:
    """Formata os resultados para o Gemini."""
    print("Formatando contexto dos resultados.")
    context = "Livros relevantes encontrados:\n\n"
    for i, result in enumerate(results):
        book_meta = metadata[result["index"]]
        context += f"**Livro {i+1}**\n"
        context += f"Título: {book_meta['titulo']}\n"
        context += f"Autor: {book_meta['autor']}\n"
        if book_meta.get('co-autor'):
            context += f"Co-autor: {book_meta['co-autor']}\n"
        context += f"Idioma: {book_meta['idioma']}\n"
        if book_meta.get('nome_comum'):
            context += f"Assuntos: {', '.join(book_meta['nome_comum'])}\n"
        if book_meta.get('pais'):
            context += f"País: {book_meta['pais']}\n"
        if book_meta.get('analitico'):
            context += f"Analítico: {book_meta['analitico']}\n"
        if book_meta.get('itype'):
            context += f"Tipo: {book_meta['itype']}\n"
        context += f"Localização: {book_meta['shelvingloc']}\n"
        context += f"Cota: {book_meta['call_no']}\n\n"
    return context

def format_history(history: List[Dict[str, str]]) -> str:
    """Formata o histórico de conversa para inclusão no prompt."""
    if not history:
        return ""
    history_text = "Histórico de conversa:\n"
    for i, entry in enumerate(history):
        history_text += f"**Pergunta {i+1}**: {entry['query']}\n"
        history_text += f"**Resposta {i+1}**: {entry['response']}\n\n"
    return history_text

def generate_gemini_response(prompt: str) -> str:
    """Gera resposta do Gemini com retry e timeout."""
    model = genai.GenerativeModel('gemini-2.0-flash-lite')
    for attempt in range(GEMINI_MAX_RETRIES):
        try:
            result = [None]
            def run_generate():
                try:
                    response = model.generate_content(prompt)
                    result[0] = response.text
                except Exception as e:
                    result[0] = f"Erro interno no Gemini: {str(e)}"

            thread = threading.Thread(target=run_generate)
            thread.start()
            thread.join(timeout=GEMINI_TIMEOUT)

            if thread.is_alive():
                print(f"Tempo limite excedido na tentativa {attempt + 1}.")
                if attempt < GEMINI_MAX_RETRIES - 1:
                    wait_time = BASE_WAIT_TIME * (2 ** attempt)
                    print(f"Aguardando {wait_time:.2f} segundos antes de tentar novamente...")
                    time.sleep(wait_time)
                    continue
                return "Erro: Tempo limite excedido após várias tentativas."
            if result[0] is None:
                print(f"Falha silenciosa na tentativa {attempt + 1}.")
                if attempt < GEMINI_MAX_RETRIES - 1:
                    time.sleep(BASE_WAIT_TIME)
                    continue
                return "Erro: Falha ao gerar resposta após várias tentativas."
            if result[0].startswith("Erro interno no Gemini"):
                print(f"Erro na tentativa {attempt + 1}: {result[0]}")
                if attempt < GEMINI_MAX_RETRIES - 1:
                    time.sleep(BASE_WAIT_TIME)
                    continue
                return result[0]
            print("Resposta gerada com sucesso.")
            return result[0]
        except Exception as e:
            print(f"Erro na tentativa {attempt + 1}: {str(e)}")
            if attempt < GEMINI_MAX_RETRIES - 1:
                time.sleep(BASE_WAIT_TIME)
            continue
    return "Erro: Não foi possível gerar resposta após várias tentativas."

@measure_execution
def answer_question(query: str, embeddings, metadata, index, history: List[Dict[str, str]], top_k: int = 5) -> str:
    """Gera resposta com base na busca, incluindo histórico de conversa."""
    print(f"Processando pergunta: {query}")
    try:
        search_results = search(query, index, metadata, top_k=top_k)
        if not search_results:
            print("Nenhum resultado encontrado.")
            response = "Não encontrei livros relevantes para sua consulta."
        else:
            context = format_context(search_results, metadata)
            print("Contexto formatado, gerando resposta com Gemini.")
            print(f"Prompt enviado ao Gemini (primeiros 500 caracteres):\n{context[:500]}...")
            history_text = format_history(history)
            prompt = f"""
            Você é um assistente de biblioteca amigável. Responda à pergunta do usuário com base no contexto abaixo e no histórico de conversa, se relevante.
            Se não houver informação suficiente, seja honesto e sugira algo útil.
            Sempre refira a localização do livro(a biblioteca onde se econtra).

            **HISTÓRICO:**
            {history_text}

            **CONTEXTO:**
            {context}

            **PERGUNTA:**
            {query}

            **RESPOSTA:**
            """
            response = generate_gemini_response(prompt)

        # Atualizar histórico
        history.append({"query": query, "response": response})
        if len(history) > MAX_HISTORY_LENGTH:
            history.pop(0)
        return response
    except Exception as e:
        print(f"Erro ao gerar resposta: {str(e)}")
        response = f"Ocorreu um erro: {str(e)}. Tente novamente ou reformule sua pergunta."
        history.append({"query": query, "response": response})
        if len(history) > MAX_HISTORY_LENGTH:
            history.pop(0)
        return response

In [ ]:
def perguntar_ao_assistente(embeddings, metadata, index):
    """Permite fazer perguntas ao assistente diretamente no console com histórico."""
    print("Bem-vindo ao Assistente de Biblioteca! Digite sua pergunta ou 'sair' para encerrar.")
    history = []
    while True:
        print("\nHistórico de conversa:")
        for i, entry in enumerate(history):
            print(f"Pergunta {i+1}: {entry['query']}")
            print(f"Resposta {i+1}: {entry['response'][:100]}...")
        pergunta = input("\nDigite sua pergunta: ")
        if pergunta.lower() == 'sair':
            print("Encerrando o assistente.")
            break
        response, metrics = answer_question(pergunta, embeddings, metadata, index, history)
        print(f"Resposta: {response}")


In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import time

In [ ]:
embeddings, metadata, index = initialize_retriever(EMBEDDINGS_DIR)


def chatbot_interface(embeddings, metadata, index):
    """Cria uma interface Gradio mínima para o assistente de biblioteca."""
    history = []

    def respond(message, chat_history):
        # Chame a função do seu chatbot para gerar a resposta
        response, metrics = answer_question(message, embeddings, metadata, index, history)

        # Simule um atraso para dar a impressão de que o chatbot está "pensando"
        for i in range(len(response)):
            time.sleep(0.01)
            yield response[: i + 1]

    demo = gr.ChatInterface(
        respond,  # Use a função modificada aqui
        type='messages',
        flagging_mode="manual",
        flagging_options=["Like", "Spam", "Inappropriate", "Other"],
        save_history=True,
    )
    return demo # Return the demo object


if __name__ == "__main__":
    demo = chatbot_interface(embeddings, metadata, index) # Call the function and assign the result to demo
    demo.launch(debug = True)

Carregando recursos de retrieval...
Embeddings carregados.
Metadados carregados.
Recursos carregados: 18266 livros indexados
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4c5d18569c76a58112.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processando pergunta: Ola. Quero saber mais sobre mineracao
Iniciando busca para: Ola. Quero saber mais sobre mineracao
Gerando embedding para: Ola. Quero saber mais sobre mineracao
Embedding gerado com sucesso.
Busca concluída no índice FAISS.
Formatando contexto dos resultados.
Contexto formatado, gerando resposta com Gemini.
Prompt enviado ao Gemini (primeiros 500 caracteres):
Livros relevantes encontrados:

**Livro 1**
Título: Conceitos de material grosseiro, mineral alterável e reserva mineral do solo : breves notas / António Augusto Guerra Réfega
Autor: Réfega, António Augusto Guerra
Idioma: Português.
Assuntos: Ciência e gestäo do solo
Localização: Escola Superior Agrária
Cota: P30-1366

**Livro 2**
Título: Produçäo e exploraçäo : aspectos tecnológicos / Albino de Carvalho
Autor: Carvalho, Albino de
Idioma: Português.
Assuntos: Produçäo florestal - Engenharia flor...
Resposta gerada com sucesso.
Tempo de execução: 2.93 segundos
Memória usada: 0.09 MB
Memória máxima: 0.36 MB


In [ ]:
# ==============================================================================
# EXECUÇÃO
# ==============================================================================
if __name__ == "__main__":
    try:
        embeddings, metadata, index = initialize_retriever(EMBEDDINGS_DIR)
        print("Escolha o modo de interação:")
        print("1. Interface Gradio")
        print("2. Console (perguntar_ao_assistente)")
        choice = input("Digite 1 ou 2: ")
        if choice == "1":
            demo = chatbot_interface(embeddings, metadata, index)
            try:
                demo.launch(share=True)
            except Exception as e:
                print(f"Erro ao iniciar o Gradio: {str(e)}. Tente o modo console ou reinicie o runtime.")
        elif choice == "2":
            perguntar_ao_assistente(embeddings, metadata, index)
        else:
            print("Escolha inválida. Encerrando.")
    except Exception as e:
        print(f"Erro ao iniciar o sistema: {str(e)}")

Carregando recursos de retrieval...
Embeddings carregados.
Metadados carregados.
Recursos carregados: 1883 livros indexados
Escolha o modo de interação:
1. Interface Gradio
2. Console (perguntar_ao_assistente)
Digite 1 ou 2: 2
Bem-vindo ao Assistente de Biblioteca! Digite sua pergunta ou 'sair' para encerrar.

Histórico de conversa:

Digite sua pergunta: Eu quero livros de mineracao
Processando pergunta: Eu quero livros de mineracao
Iniciando busca para: Eu quero livros de mineracao
Gerando embedding para: Eu quero livros de mineracao
Embedding gerado com sucesso.
Busca concluída no índice FAISS.
Formatando contexto dos resultados.
Contexto formatado, gerando resposta com Gemini.
Prompt enviado ao Gemini (primeiros 500 caracteres):
Livros relevantes encontrados:

**Livro 1**
Título: Ensino da geologia : perspectivas científicas / Miguel Telles Antunes
Autor: Antunes, Miguel Telles
Idioma: Português.
Assuntos: Geologia - ensino das ciências da terra - formação de professores em exercíci

In [ ]:
def chatbot_interface(embeddings, metadata, index):
    """Cria uma interface Gradio mínima para o assistente de biblioteca."""
    history = []

    def respond(message, chat_history):
        if not message.strip():
            chat_history.append([message, "Por favor, digite uma pergunta."])
            return chat_history
        try:
            response, metrics = answer_question(message, embeddings, metadata, index, history)
            history.append({"query": message, "response": response})
            if len(history) > MAX_HISTORY_LENGTH:
                history.pop(0)
            chat_history.append([message, f"{response}\n\nMétricas: Tempo={metrics['execution_time']:.2f}s, Memória={metrics['memory_used']:.2f}MB"])
            return chat_history
        except Exception as e:
            print(f"Erro: {str(e)}")
            chat_history.append([message, f"Erro: {str(e)}"])
            return chat_history

    iface = gr.Interface(
        fn=respond,
        inputs=[
            gr.Textbox(placeholder="Ex.: Livros sobre biodiversidade florestal", label="Pergunta"),
            gr.Chatbot(label="Conversa")
        ],
        outputs=[
            gr.Chatbot(label="Conversa")
        ],
        title="Assistente de Biblioteca",
        description="Pesquise livros na biblioteca acadêmica.",
        allow_flagging="never"
    )
    return iface